In [0]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=25ad932d5b93b6ff7415c4338b54c3ef317ed6b090e1eee241444ddaa0131fa5
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:

from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC,SVR
from sklearn import svm
from collections import Counter #for Smote, 
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
#add following to your code
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, f_classif

import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
trainfile = r'/gdrive/My Drive/2019fall/cis508/TeamProject/Preprocess_Train.csv'
Train_Data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/2019fall/cis508/TeamProject/Preprocess_Test.csv'
Test_Data = pd.read_csv(testfile)

In [0]:
Train_Data.dtypes

Unnamed: 0         int64
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice        float64
Length: 77, dtype: object

In [0]:
Test_Data.dtypes

Unnamed: 0         int64
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 76, dtype: object

In [0]:
# Log transform the target for official scoring
Train_Data['SalePrice'] = np.log1p(Train_Data['SalePrice'])
y_Train = Train_Data["SalePrice"]
print(Train_Data.shape)

(1460, 77)


In [0]:
#Copy Train data excluding target
#categoricalFeatures = Train_Data.columns

trainData_Copy = Train_Data.iloc[:, :-1].copy()
testData_Copy = Test_Data.iloc[:, :].copy()

trainData_Copy.head()

#combine data together 
combined_Data = pd.concat([trainData_Copy, testData_Copy], keys=[0,1])
#select categorical feature then we are going to dummy them
categoricalFeatures = combined_Data.select_dtypes(exclude=['int','float64']).columns
#dummy variables
combined_Data = pd.get_dummies(combined_Data, columns = categoricalFeatures)
#save feature names because we are going to do rescaler
combined_Data_col = combined_Data.columns

#Separate Train data and test data
X_Train = combined_Data.xs(0)
X_Test = combined_Data.xs(1)

#Normalize for train data
Xscaler = StandardScaler()
xscaled = Xscaler.fit(X_Train)
XscalerTrans = Xscaler.transform(X_Train)
#give feature names back which we saved before
X_Train = pd.DataFrame(XscalerTrans, columns=combined_Data_col)
print(X_Train.head())
print(X_Train.shape)

#Normalize for test data
xscaled = Xscaler.fit(X_Test)
XscalerTrans = Xscaler.transform(X_Test)
#give feature names back which we saved before
X_Test = pd.DataFrame(XscalerTrans, columns=combined_Data_col)
print(X_Test.head())
print(X_Test.shape)

   Unnamed: 0        Id  ...  SaleCondition_Normal  SaleCondition_Partial
0   -1.730865 -1.730865  ...              0.467651              -0.305995
1   -1.728492 -1.728492  ...              0.467651              -0.305995
2   -1.726120 -1.726120  ...              0.467651              -0.305995
3   -1.723747 -1.723747  ...             -2.138345              -0.305995
4   -1.721374 -1.721374  ...              0.467651              -0.305995

[5 rows x 272 columns]
(1460, 272)
   Unnamed: 0        Id  ...  SaleCondition_Normal  SaleCondition_Partial
0   -1.730864 -1.730864  ...              0.460211              -0.299365
1   -1.728490 -1.728490  ...              0.460211              -0.299365
2   -1.726115 -1.726115  ...              0.460211              -0.299365
3   -1.723741 -1.723741  ...              0.460211              -0.299365
4   -1.721367 -1.721367  ...              0.460211              -0.299365

[5 rows x 272 columns]
(1459, 272)


In [0]:

X_train, X_test, y_train, y_test = train_test_split(X_Train, y_Train , test_size=0.2, random_state=10)
print(X_train.shape)
print (Test_Data.shape)

(1168, 272)
(1459, 76)


In [0]:
#Support Vector Regressor================================================================================
parameters = {'kernel' : ['rbf']}
svr = SVR()
svr_random = RandomizedSearchCV(svr,parameters, n_iter = 15)
svr_random.fit(X_train, y_train)
grid_parm_svr=svr_random.best_params_
print(grid_parm_svr)

#Randomized Search for hyperparameter tuning
svr = SVR(**grid_parm_svr)
svr.fit(X_train, y_train) 
svr_predict_Train=svr.predict(X_train)
#svr 0.21
mean_squared_error(y_train,svr_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,svr_predict_Train)))
svr_predict_Test=svr.predict(X_test)
mean_squared_error(y_test,svr_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,svr_predict_Test)))


{'kernel': 'rbf'}
RMSE (training) for Decision Tree:  0.005957
RMSE (Test Data) for Decision Tree:  0.037538


In [0]:
pred_test = pd.DataFrame(svr.predict(X_Test), columns=["SalePrice"])
pred_test = np.exp(pred_test)

result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/SVR.csv", index = False)

In [0]:
#Support Vector Regressor================================================================================
parameters = {'kernel' : ['rbf']}
svr = SVR()
svr_random = RandomizedSearchCV(svr,parameters, n_iter = 15)
svr_random.fit(X_train, y_train)
grid_parm_svr=svr_random.best_params_
print(grid_parm_svr)

#Randomized Search for hyperparameter tuning
svr = SVR(**grid_parm_svr)
svr.fit(X_train, y_train) 
svr_predict_Train=svr.predict(X_train)

mean_squared_error(y_train,svr_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,svr_predict_Train)))
svr_predict_Test=svr.predict(X_test)
mean_squared_error(y_test,svr_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,svr_predict_Test)))


{'kernel': 'rbf'}
RMSE (training) for Decision Tree:  0.005957
RMSE (Test Data) for Decision Tree:  0.037538


In [0]:
pred_test = pd.DataFrame(svr.predict(X_Test), columns=["SalePrice"])
pred_test = np.exp(pred_test)

result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/SVRht.csv", index = False)

In [0]:
#Random Forest Regressor==============================================================================
#=================================================================================================

rfc = RandomForestRegressor()
rfc.fit(X_train, y_train)
rfc_predict_Train=rfc.predict(X_train)

mean_squared_error(y_train,rfc_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,rfc_predict_Train)))
rfc_predict_Test=rfc.predict(X_test)
mean_squared_error(y_test,rfc_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,rfc_predict_Test)))

RMSE (training) for Decision Tree:  0.004533
RMSE (Test Data) for Decision Tree:  0.023071


In [0]:
pred_test = pd.DataFrame(rfc.predict(X_Test), columns=["SalePrice"])
pred_test = np.exp(pred_test)

result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/RFbasic.csv", index = False)

In [0]:

#Hyperparameter tuning for random forest
parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(3,30,3),'n_estimators':[20,30,40]}
rfc = RandomForestRegressor()
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc = RandomForestRegressor(**grid_parm_rfc)
rfc.fit(X_train, y_train)
rfc_predict_Train=rfc.predict(X_train)
#random foerest 0.18

mean_squared_error(y_train,rfc_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,rfc_predict_Train)))
rfc_predict_Test=rfc.predict(X_test)
mean_squared_error(y_test,rfc_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,rfc_predict_Test)))

{'n_estimators': 40, 'min_samples_leaf': 10, 'max_depth': 15}
RMSE (training) for Decision Tree:  0.014776
RMSE (Test Data) for Decision Tree:  0.025158


In [0]:
pred_test = pd.DataFrame(rfc.predict(X_Test),columns=["SalePrice"])
pred_test = np.exp(pred_test)
result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/RFht.csv", index = False)

In [0]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf = DecisionTreeRegressor()
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeRegressor(**grid_parm)
clf.fit(X_train, y_train)
clf_predict_Train=clf.predict(X_train)


#clf.feature_importances_
mean_squared_error(y_train,clf_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,clf_predict_Train)))
clf_predict_Test=clf.predict(X_test)
mean_squared_error(y_test,clf_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,clf_predict_Test)))


{'min_samples_split': 30, 'max_depth': 13}
RMSE (training) for Decision Tree:  0.013547
RMSE (Test Data) for Decision Tree:  0.034983


In [0]:
pred_test = pd.DataFrame(clf.predict(X_Test),columns=["SalePrice"])
pred_test = np.exp(pred_test)
result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/DCht.csv", index = False)

In [0]:
#Gradient Boosting Regressor================================================================================

abc = GradientBoostingRegressor()
abc.fit(X_train, y_train)
abc_predict_Train=abc.predict(X_train)

mean_squared_error(y_train,abc_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,abc_predict_Train)))
abc_predict_Test=abc.predict(X_test)
mean_squared_error(y_test,abc_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,abc_predict_Test)))

RMSE (training) for Decision Tree:  0.005653
RMSE (Test Data) for Decision Tree:  0.017245


In [0]:
pred_test = pd.DataFrame(abc.predict(X_Test),columns=["SalePrice"])
pred_test = np.exp(pred_test)
result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/GBbasic.csv", index = False)

In [0]:


#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
abc =GradientBoostingRegressor()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)


#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)


mean_squared_error(y_train,abc_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,abc_predict_Train)))
abc_predict_Test=abc.predict(X_test)
mean_squared_error(y_test,abc_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,abc_predict_Test)))

{'n_estimators': 50, 'learning_rate': 0.1}
RMSE (training) for Decision Tree:  0.005653
RMSE (Test Data) for Decision Tree:  0.017376


In [0]:
pred_test = pd.DataFrame(abc.predict(X_Test),columns=["SalePrice"])
pred_test = np.exp(pred_test)
result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/GBht.csv", index = False)

In [0]:
#MLP Regressor================================================================================

mlp = MLPRegressor()
mlp.fit(X_train, y_train) 
mlp_predict_Train=mlp.predict(X_train)

mean_squared_error(y_train,abc_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,mlp_predict_Train)))
mlp_predict_Test=mlp.predict(X_test)
mean_squared_error(y_test,mlp_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,mlp_predict_Test)))

RMSE (training) for Decision Tree:  0.010187
RMSE (Test Data) for Decision Tree:  7.715640


In [0]:
pred_test = pd.DataFrame(mlp.predict(X_Test),columns=["SalePrice"])
pred_test = np.exp(pred_test)
result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/MLP.csv", index = False)

In [0]:
#Hyperparameter tuning 
parameters={'solver' :['lbfgs']}
mlp = MLPRegressor()
mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=15)
mlp_random.fit(X_train, y_train)
grid_parm_mlp=mlp_random.best_params_
print(grid_parm_mlp)

mlp = MLPRegressor(**grid_parm_mlp)
mlp.fit(X_train, y_train)
mlp_predict_Train=mlp.predict(X_train)

mean_squared_error(y_train,mlp_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,mlp_predict_Train)))
mlp_predict_Test=mlp.predict(X_test)
mean_squared_error(y_test,mlp_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,mlp_predict_Test)))



{'solver': 'lbfgs'}
RMSE (training) for Decision Tree:  0.000014
RMSE (Test Data) for Decision Tree:  3.193813


In [0]:
mlp_predict_Train.shape

(1168,)

In [0]:
pred_test = pd.DataFrame(mlp.predict(X_Test),columns=["SalePrice"])
pred_test = np.exp(pred_test)
result = pd.concat([Test_Data['Id'], pred_test], axis=1)
result.to_csv("/gdrive/My Drive/2019fall/cis508/TeamProject/MLPht.csv", index = False)

In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using MLP, Support Vector Regressor GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingRegressor(), RandomForestRegressor(), DecisionTreeRegressor(), MLPRegressor(), SVR()]
      
S_Train, S_Test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
                                        
                           n_folds=5, 
                                                    
                           verbose=2)



___________________________________________________________________________________________
Ensemble Methods Predictions using MLP, Support Vector Regressor GradientBoosting, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [GradientBoostingRegressor]
    fold  0:  [0.08691073]
    fold  1:  [0.09129255]
    fold  2:  [0.08527911]
    fold  3:  [0.09220537]
    fold  4:  [0.08720627]
    ----
    MEAN:     [0.08857881] + [0.00268592]
    FULL:     [0.08857688]

model  1:     [RandomForestRegressor]
    fold  0:  [0.10680607]
    fold  1:  [0.11037845]
    fold  2:  [0.11122587]
    fold  3:  [0.10874031]
    fold  4:  [0.10220218]
    ----
    MEAN:     [0.10787058] + [0.00321147]
    FULL:     [0.10787468]

model  2:     [DecisionTreeRegressor]
    fold  0:  [0.14489166]
    fold  1:  [0.16347226]
    fold  2:  [0.13691445]
    fold  3:  [0.16692748]
    fold  4:  [0.

In [0]:
#STACKING - CONTRUCT A Decision Tree Classifier==============================
model = DecisionTreeRegressor()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))

RMSE (training) for Decision Tree:  0.000000
RMSE (Test Data) for Decision Tree:  0.020990


In [0]:
#STACKING - CONTRUCT A Random Forest Regressor==============================
model = RandomForestRegressor()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))

RMSE (training) for Decision Tree:  0.003043
RMSE (Test Data) for Decision Tree:  0.017410


In [0]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingRegressor()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))


RMSE (training) for Decision Tree:  0.008660
RMSE (Test Data) for Decision Tree:  0.016398


In [0]:
#STACKING - CONTRUCT A SVR==============================
model = SVR()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))

RMSE (training) for Decision Tree:  0.015774
RMSE (Test Data) for Decision Tree:  0.018896


In [0]:
#MLPRegressor
#STACKING - CONTRUCT A MLPRegressor==============================
model = MLPRegressor()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))

RMSE (training) for Decision Tree:  0.022392
RMSE (Test Data) for Decision Tree:  0.024072
